Data preprocessing

In [ ]:
import pandas as pd

df_data = pd.read_csv('./medicine_project/rawData/DATA_Hosp_ver.csv')

# 濾掉空格
df_data['drug_no'] = df_data['drug_no'].str.replace(' ', '')
df_data['acode_icd9_1'] = df_data['acode_icd9_1'].str.replace(' ', '')

# 濾掉非服用藥的資料
df_data = df_data[df_data['drug_no'].str.len() == 10]

df_data = df_data[df_data['acode_icd9_1'].isin(df_dx['icd9_cm_code'])]

df_data.to_csv('clear_data.csv', index = False)

In [ ]:
import pandas as pd
df = pd.read_csv('./medicine_project/rawData/DXREF_2008_Archive.csv')
df_dx.columns = df_dx.columns.str.replace('\'', '')
df_dx['ICD-9-CM CODE'] = df_dx['ICD-9-CM CODE'].str.replace('\'', '')
df_dx['ICD-9-CM CODE'] = df_dx['ICD-9-CM CODE'].str.replace(' ', '')
df_dx['CCS CATEGORY'] = df_dx['CCS CATEGORY'].str.replace('\'', '')
df_dx['CCS CATEGORY'] = df_dx['CCS CATEGORY'].str.replace(' ', '')
df_dx['CCS CATEGORY DESCRIPTION'] = df_dx['CCS CATEGORY DESCRIPTION'].str.replace('\'', '')
df_dx.columns = ['icd9_cm_code', 'ccs_category', 'ccs_category_description', 'icd9_cm_code_description']
df_dx['ccs_category'] = df_dx['ccs_category'].astype(int)
df_dx.sort_values(by = 'ccs_category', inplace = True)
df_dx = df_dx[df_dx['ccs_category'] < 1000]
df_dx.to_csv('dxref.csv', index = False)

In [ ]:
df_A = pd.read_csv('./all1_1080108_A.csv')
df_A = df_A[['B', 'G']]
df_B = pd.read_csv('./all1_1080108_B.csv')
df_B = df_B[['B', 'F']]
df_A.columns = df_B.columns = ['drug_no', 'drug_name']
df_all = pd.concat([df_A, df_B], ignore_index = True)
df_all = df_all.sort_values(by = 'drug_no')
df_drugs = df_all.drop_duplicates('drug_no')
df_drugs = df_drugs[df_drugs['drug_no'].str.len() == 10] #去掉 YC00004217 1 這種

df_drugs.to_csv('drugs.csv', index = False)

In [ ]:
df_data = pd.read_csv('./clear_data.csv')
df_dx = pd.read_csv('./dxref.csv')
df_dr = pd.read_csv('./drugs_new.csv')

df_result = pd.DataFrame(columns = ['icd9_cm_code', 'ccs_category', 'ccs_description', 'drug_no', 'drug_name', 'frequency'])

ccs_memory = {}
name_memory = {}

count = 0
for _ in df_data.index:
    
    if df_data['acode_icd9_1'][_] in ccs_memory:
        df_ccs = ccs_memory[df_data['acode_icd9_1'][_]]
    else:
        df_ccs = df_dx[df_dx['icd9_cm_code'].isin([df_data['acode_icd9_1'][_]])]
        ccs_memory[df_data['acode_icd9_1'][_]] = df_ccs
    
    if df_data['drug_no'][_] in name_memory:
        df_name = name_memory[df_data['drug_no'][_]]
    else:
        df_name = df_dr[df_dr['drug_no'].isin([df_data['drug_no'][_]])]
        name_memory[df_data['drug_no'][_]] = df_name
        
    df_result.loc[_] = [df_data['acode_icd9_1'][_], df_ccs['ccs_category'].values[0], df_ccs['ccs_category_description'].values[0], df_data['drug_no'][_], df_name['drug_name'].values, '0']

    count += 1
    if count > 1000:
        print(df_result)
        count = 0

df_result.to_csv('first_result.csv', index = False)

In [ ]:
import pandas as pd

df = pd.read_csv('./first_result.csv')
df.sort_values(by = ['ccs_category', 'drug_no'], inplace = True)
df.to_csv('fixed.csv', index = False)

ccs_cat = df['ccs_description'].drop_duplicates().values
for _ in ccs_cat:
    print(_)
    df_tmp = df.loc[df['ccs_description'] == _]
    drug_arr = df_tmp['drug_no'].drop_duplicates().values
    for __ in drug_arr:
        count = list(df_tmp['drug_no']).count(__)
#         df_tmp.loc[df['drug_no'] == __].drop_duplicates(inplace = True) 沒有效果，不知如何更改
        df_tmp.loc[df['drug_no'] == __, 'frequency'] = count
    df.loc[df['ccs_description'] == _] = df_tmp

df.to_csv('final_result', index = False)